# Transfer-learning tutorial using DenseNet-121 pre-trained model:
# example on MedNIST dataset


## Goal of this tutoriel

This tutorial shows how to do 2d images classification example on MedNIST dataset using pretrained PyTorch model.

The goal of this tutorial is to provide an example of transfer learning methods with Fed-BioMed for medical images classification.

## About the model

The model used is Densenet-121 model(“Densely Connected Convolutional Networks”) pretrained on ImageNet dataset. The Pytorch pretrained model [Densenet121](https://pytorch.org/vision/main/models/generated/torchvision.models.html). to perform image classification on the MedNIST dataset. 
The goal of this Densenet121 model is to predict the class of medical images.



### About MedNIST

MedNIST provides an artificial 2d classification dataset created by gathering different medical imaging datasets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.

MedNIST dataset is downloaded from the resources provided by the project MONAI: https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/MedNIST.tar.gz

The dataset MedNIST has 58954 images of size (3, 64, 64) distributed into 6 classes (10000 images per class except for BreastMRI class which has 8954 images). Classes are AbdomenCT, BreastMRI, CXR, ChestCT, Hand, HeadCT. It has the structure:

└── MedNIST/

    ├── AbdomenCT/

    └── BreastMRI/

    └── CXR/

    └── ChestCT/

    └── Hand/

    └── HeadCT/   
   

### Transfer-learning
Transfer learning is a machine learning technique where a model trained on one task is repurposed or adapted for a second related task. Transfer learning uses a pre-trained neural network on a large dataset, as Imagenet is used to train DenseNet model to perform classification of a wide diversity of images.

The objective is that the knowledge gained from learning one task can be useful for learning another task (as we do here, classification of medical images in 6 categories). This is particularly beneficial when the amount of labeled data for the target task is limited, as the pre-trained model has already learned useful features and representations from a large dataset.

Transfer learning is typically applied in one of two ways:

- (I) Feature Extraction: In this approach, the pre-trained model is used as a fixed feature extractor. The earlier layers of the neural network, which capture general features and patterns, are frozen, and only the later layers are replaced or retrained for the new task. 

- (II) Fine-tuning: In this approach, the pre-trained model is further trained or partially trained on the new task. This allows the model to adapt its learned representations to the specifics of the new task while retaining some of the knowledge gained from the original task.


In this example, we load on the node a sampled dataset ( 500 or 1000 images) of MedNIST to illustrate the effectiveness of the transfer learning. The sampled dataset is made with a random selection of images and return a sampled dataset with balanced classes, to avoid classification's bias.
We will test these two approches through two independant TrainingPlan experiments. 
To illustrate the effectiveness of these two method, we load 500 images for the first experiment and 1000 images for the second. Because the fine tunng method involves more layers's training, this method is better efficient for large datatsets. 

### 1. Load dataset or sampled dataset
- From the root directory of Fed-BioMed, run :  source ./scripts/fedbiomed_environment node in order to load the Node environment
- If you have already ran Mednist nodes before, clean remaining MedNIST nodes : run ./scripts/fedbiomed_run node delete or source ./scripts/fedbiomed_environment clean
- In this new environment, run the script python: python ./notebooks/transfer-learning/download_sample_of_mednist.py -n <number-of-nodes>, with <number-of-nodes> the number of Nodes you want to create( for more details about this script, please run notebooks/transfer-learning/download_sample_of_mednist.py --help)
- The script will ask for each Nodes created the number of samples you want for your dataset. Scripts will output configuration files for each of Nodes, with configured database.  
- Finally launch your Nodes (one by terminal) by running: ./scripts/fedbiomed_run node config  start config_mednist_<i>_sampled.ini start, where <i> corresponds to the number of Node created.  Wait until you get Starting task manager.

### 2. Launch the researcher 
- From the root directory of Fed-BioMed, run : ./scripts/fedbiomed_run researcher start
- It opens the Jupyter notebook.

To make sure that MedNIST dataset is loaded in the node we can send a request to the network to list the available dataset in the node. The list command should output an entry for mednist data.

 

In [53]:
from fedbiomed.researcher.requests import Requests
req  = Requests()
req.list()

2024-02-06 13:46:44,058 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 13:46:44,059 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

{'config_mednist_1_sampled': [{'name': 'MedNIST_1_sampled',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MedNIST dataset for transfer learning',
   'shape': [500, 3, 64, 64],
   'dataset_id': 'dataset_29462eb2-6c03-40a4-adde-e9c5a5b2a3c8',
   'dataset_parameters': None}],
 'config_mednist_2_sampled': [{'name': 'MedNIST_2_sampled',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MedNIST dataset for transfer learning',
   'shape': [1000, 3, 64, 64],
   'dataset_id': 'dataset_dded8492-1797-4cc6-8057-26bf1fe437d7',
   'dataset_parameters': None}]}

## Import of librairies 

In [54]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan

from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage


## Run an expriment for image's classification using Transfer-learning 

### I- Adapt the last layer to your classification's goal
Here we use the DenseNet model that allows classification through 10000 classes. 
We could adapt this classification's task to the MedNIST dataset by replacing the last layer with our classifier. 
The model.classifier classify images through 6 classes, by adapting the num_classes value. 

### Data augmentation
You could perform data augmentation through the preprocess part if you need. Here I show random flip, rotation and crops. 
You could do the preprocessing of images by doing only transforms.resize, transforms.to_tensor and transforms.normalize, as mentionned in the code below. 

### 1. Define Training plan experiment 

In [55]:
class MyTrainingPlan1(TorchTrainingPlan):

    def init_model(self, model_args):
       
        # Load the pre-trained DenseNet model
        model = models.densenet121(pretrained=True)
        
        # Remove the classification layer of DenseNet
        for param in model.features[:-1].parameters():
            param.requires_grad = False
            
        # add the classifier 
        num_classes = model_args['num_classes'] 
        num_ftrs = model.classifier.in_features
        model.classifier= nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
      
        return model

    def init_dependencies(self):
        return [
            "from torchvision import datasets, transforms, models",
            "import torch.optim as optim",
            "from torchvision.models import densenet121"
        ]


    def init_optimizer(self, optimizer_args):        
        return optim.Adam(self.model().parameters(), lr=optimizer_args["lr"])

    
    # training data
    
    def training_data(self):
        
        # Custom torch Dataloader for MedNIST data
        print("dataset path",self.dataset_path)

        # Transform images and  do data augmentation 
        preprocess = transforms.Compose([
                transforms.Resize((224,224)),  
                #transforms.RandomHorizontalFlip(p=0.5),
                #transforms.RandomVerticalFlip(p=0.5),
                #transforms.RandomRotation(30),
                #transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
           ])
    
        train_data = datasets.ImageFolder(self.dataset_path,transform = preprocess)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=train_data, **train_kwargs)

    def training_step(self, data, target):
        output = self.model().forward(data)
        loss_func = nn.CrossEntropyLoss()
        loss   = loss_func(output, target)
        return loss




In [56]:
training_args = {
    'loader_args': { 'batch_size': 32, }, 
    'optimizer_args': {'lr': 1e-3}, 
    'epochs': 2, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

model_args = {
    'num_classes': 6 # adapt this number to the number of classes in your dataset
}

In [57]:
tags =  ['#MEDNIST', '#dataset']

rounds = 2 # adjsut the number of rounds 

exp = Experiment(tags=tags,
                 training_plan_class=MyTrainingPlan1,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage())

# testing section 
from fedbiomed.common.metrics import MetricTypes
exp.set_test_ratio(.1) 
exp.set_test_on_local_updates(True)
exp.set_test_metric(MetricTypes.ACCURACY)

exp.set_tensorboard(True)

2024-02-06 13:46:50,953 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 13:46:50,955 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 13:46:50,958 fedbiomed INFO - Node selected for training -> config_mednist_1_sampled

2024-02-06 13:46:50,959 fedbiomed INFO - Node selected for training -> config_mednist_2_sampled

2024-02-06 13:46:50,961 fedbiomed DEBUG - Model file has been saved: /home/ebirgy/development/fedbiomed_github/fedbiomed/var/experiments/Experiment_0008/model_1a17ef84-490c-4681-998a-5b667dadb1e3.py

...

2024-02-06 13:46:51,102 fedbiomed INFO - Removing tensorboard logs from previous experiment

2024-02-06 13:46:51,105 fedbiomed DEBUG - Experimentation training_args updated for `job`

2024-02-06 13:46:51,106 fedbiomed DEBUG - Experimentation training_args updated for `job`

2024-02-06 13:46:51,107 fedbiomed DEBUG - Experimentation training_args updated for `job`

True

### 2. Define the dataset foryour experiment 

I propose to run these first experiment with only MedNIST_sampled_1 dataset, the dataset of 500 images, because these first method is a transfer learning without training.
Here I show how to select one dataset :

In [58]:

exp.set_nodes(['config_mednist_1_sampled'])
exp.set_tags(['#MEDNIST', '#dataset'])
td = exp.training_data().data()
td.pop('config_mednist_2_sampled')
exp.set_training_data(td)

print(exp.training_data().data())

2024-02-06 13:46:58,682 fedbiomed DEBUG - Experimentation nodes filter changed, you may need to update `training_data`

2024-02-06 13:46:58,684 fedbiomed DEBUG - Experimentation tags changed, you may need to update `training_data`

2024-02-06 13:46:58,685 fedbiomed DEBUG - Training data changed, you may need to update `node_selection_strategy`

2024-02-06 13:46:58,685 fedbiomed DEBUG - Training data changed, you may need to update `job`

2024-02-06 13:46:58,686 fedbiomed DEBUG - Training data changed, you may need to update `aggregator`

{'config_mednist_1_sampled': {'name': 'MedNIST_1_sampled', 'data_type': 'mednist', 'tags': ['#MEDNIST', '#dataset'], 'description': 'MedNIST dataset for transfer learning', 'shape': [500, 3, 64, 64], 'dataset_id': 'dataset_29462eb2-6c03-40a4-adde-e9c5a5b2a3c8', 'dtypes': [], 'dataset_parameters': None}}


### 3. Run your experiment 

In [59]:
exp.run()

2024-02-06 13:47:08,524 fedbiomed INFO - Sampled nodes in round 0 ['config_mednist_1_sampled']

2024-02-06 13:47:08,533 fedbiomed INFO - Sending request 
					 To: config_mednist_1_sampled 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-02-06 13:47:08,671 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 13:47:10,481 fedbiomed INFO - TRAINING 
					 NODE_ID: config_mednist_1_sampled 
					 Round 1 Epoch: 1 | Iteration: 1/15 (7%) | Samples: 32/480
 					 Loss: 1.770158 
					 ---------

2024-02-06 13:47:16,019 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

...

2

2024-02-06 13:49:27,838 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 13:49:27,896 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 13:49:36,411 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 13:49:42,351 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

###### For example,  At the end of training experiment, I obtained

                      INFO - VALIDATION ON LOCAL UPDATES 
					 NODE_ID: NODE_41cd99c8-3571-4ab3-958e-6357ce31e91b 
					 Round 2 | Iteration: 1/1 (100%) | Samples: 100/100
 					 ACCURACY: 0.980000
					 -

### Save your model 
You could save your model to use it in a new TrainingPlan 
This save allow to import the model including your layers's modification and weights values.

In [70]:
#save model 
exp.training_plan().export_model('./training_plan1_densenet_MedNIST')

### Results in tensorboard 

In [71]:
from fedbiomed.researcher.environ import environ
tensorboard_dir = environ['TENSORBOARD_RESULTS_DIR']

In [72]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [73]:
%tensorboard --logdir "$tensorboard_dir"

Reusing TensorBoard on port 6006 (pid 45074), started 2:23:43 ago. (Use '!kill 45074' to kill it.)

## II - Partial fine-tuning: Use pretrained DenseNet and train specific layers with your data
You can set the second dataset with more images to run the second experiment that uses training steps. 

In this example, I run a second experiment with 1000 images.
The dataset is defined below, after TrainingPlan as previously shown.

You could also import the model you saved to perform your second TrainingPlan experiment (let's see below)


In [76]:
from fedbiomed.researcher.requests import Requests
req  = Requests()
req.list()

2024-02-06 14:04:38,957 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:04:38,961 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

{'config_mednist_1_sampled': [{'name': 'MedNIST_1_sampled',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MedNIST dataset for transfer learning',
   'shape': [500, 3, 64, 64],
   'dataset_id': 'dataset_29462eb2-6c03-40a4-adde-e9c5a5b2a3c8',
   'dataset_parameters': None}],
 'config_mednist_2_sampled': [{'name': 'MedNIST_2_sampled',
   'data_type': 'mednist',
   'tags': ['#MEDNIST', '#dataset'],
   'description': 'MedNIST dataset for transfer learning',
   'shape': [1000, 3, 64, 64],
   'dataset_id': 'dataset_dded8492-1797-4cc6-8057-26bf1fe437d7',
   'dataset_parameters': None}]}

In [85]:
from fedbiomed.common.training_plans import TorchTrainingPlan
class MyTrainingPlan2(TorchTrainingPlan):

    def init_model(self, model_args):

        # Load the pre-trained DenseNet model
        model = models.densenet121(pretrained=True)
        
        # For example, let's freeze layers of the last dense block
        for param in model.features[:-3].parameters():
            param.requires_grad = False

        # add the classifier 
        num_ftrs = model.classifier.in_features
        num_classes = model_args['num_classes'] 
        model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)       
            )
        
        return model

    def init_dependencies(self):
        return [
            "from torchvision import datasets, transforms, models",
            "import torch.optim as optim"
        ]


    def init_optimizer(self, optimizer_args):        
        return optim.Adam(self.model().parameters(), lr=optimizer_args["lr"])

    def training_data(self):
        
        # Custom torch Dataloader for MedNIST data and transform images and perform data augmentation 
        print("dataset path",self.dataset_path)
        preprocess = transforms.Compose([
                transforms.Resize((224,224)),  
                #transforms.RandomHorizontalFlip(p=0.5),
                #transforms.RandomVerticalFlip(p=0.5),
                #transforms.RandomRotation(30),
                #transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
           ])
        train_data = datasets.ImageFolder(self.dataset_path,transform = preprocess)
        train_kwargs = { 'shuffle': True}
        return DataManager(dataset=train_data, **train_kwargs)



    def training_step(self, data, target):
        output = self.model().forward(data)
        loss_func = nn.CrossEntropyLoss()
        loss   = loss_func(output, target)
        return loss




In [86]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

training_args = {
    'loader_args': { 'batch_size': 32, }, 
    'optimizer_args': {'lr': 1e-4}, # You could decrease the learning rate
    'epochs': 1, # you can increase the epoch's number =10
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}
model_args={
    'num_classes': 6
}
tags =  ['#MEDNIST', '#dataset']
rounds = 1  # you can increase the rounds's number 

exp = Experiment(tags=tags,
                 training_plan_class=MyTrainingPlan2,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage())

from fedbiomed.common.metrics import MetricTypes
exp.set_test_ratio(.1)
exp.set_test_on_local_updates(True)
exp.set_test_metric(MetricTypes.ACCURACY)

exp.set_tensorboard(True)
    

2024-02-06 14:10:30,692 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:10:30,695 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 14:10:30,699 fedbiomed INFO - Node selected for training -> config_mednist_1_sampled

2024-02-06 14:10:30,700 fedbiomed INFO - Node selected for training -> config_mednist_2_sampled

2024-02-06 14:10:30,704 fedbiomed DEBUG - Model file has been saved: /home/ebirgy/development/fedbiomed_github/fedbiomed/var/experiments/Experiment_0011/model_d892d622-8aa9-46d3-8801-2d6f5014860f.py

...

2024-02-06 14:10:30,847 fedbiomed INFO - Removing tensorboard logs from previous experiment

2024-02-06 14:10:30,849 fedbiomed DEBUG - Experimentation training_args updated for `job`

2024-02-06 14:10:30,850 fedbiomed DEBUG - Experimentation training_args updated for `job`

2024-02-06 14:10:30,851 fedbiomed DEBUG - Experimentation training_args updated for `job`

True

### 1- Import a "custom model" or continue with the original DenseNet model of the TrainingPlan 

In [87]:
exp.training_plan().import_model('./training_plan1_densenet_MedNIST') # if you want to import the model and weights of first traning plan 

### 2. Define the dataset foryour experiment 

In [88]:
exp.set_nodes(['config_mednist_2_sampled'])
exp.set_tags(['#MEDNIST', '#dataset'])
td = exp.training_data().data()
td.pop('config_mednist_1_sampled')
exp.set_training_data(td)

print(exp.training_data().data())

2024-02-06 14:10:42,357 fedbiomed DEBUG - Experimentation nodes filter changed, you may need to update `training_data`

2024-02-06 14:10:42,360 fedbiomed DEBUG - Experimentation tags changed, you may need to update `training_data`

2024-02-06 14:10:42,362 fedbiomed DEBUG - Training data changed, you may need to update `node_selection_strategy`

2024-02-06 14:10:42,363 fedbiomed DEBUG - Training data changed, you may need to update `job`

2024-02-06 14:10:42,365 fedbiomed DEBUG - Training data changed, you may need to update `aggregator`

{'config_mednist_2_sampled': {'name': 'MedNIST_2_sampled', 'data_type': 'mednist', 'tags': ['#MEDNIST', '#dataset'], 'description': 'MedNIST dataset for transfer learning', 'shape': [1000, 3, 64, 64], 'dataset_id': 'dataset_dded8492-1797-4cc6-8057-26bf1fe437d7', 'dtypes': [], 'dataset_parameters': None}}


### 3. Run your experiment 

In [89]:
exp.run()

2024-02-06 14:10:46,450 fedbiomed INFO - Sampled nodes in round 0 ['config_mednist_2_sampled']

2024-02-06 14:10:46,459 fedbiomed INFO - Sending request 
					 To: config_mednist_2_sampled 
					 Request: : TRAIN
 -----------------------------------------------------------------

2024-02-06 14:10:46,589 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 14:10:48,622 fedbiomed INFO - TRAINING 
					 NODE_ID: config_mednist_2_sampled 
					 Round 1 Epoch: 1 | Iteration: 1/29 (3%) | Samples: 32/928
 					 Loss: 0.110120 
					 ---------

2024-02-06 14:11:03,299 fedbiomed INFO - TRAINING 
					 NODE_ID: config_mednist_2_sampled 
					 Round 1 Epoch: 1 | Iteration: 10/29 (34%) | Samples: 320/928
 					 Loss: 0.076812 
					 ---------

...

2024-02-06 14:13:15,319 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:13:27,035 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:13:36,533 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 14:13:51,549 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 14:14:27,038 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

For example,  At the end of training experiment, I obtained

                    fedbiomed INFO - VALIDATION ON LOCAL UPDATES 
					 NODE_ID: NODE_7842724a-cafa-49cc-862d-149288bbbb22 
					 Round 1 | Iteration: 1/1 (100%) | Samples: 100/100
 					 ACCURACY: 1.00000
					 ---------

In [81]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1]
for r in round_data.values():
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
        rtraining = r['timing']['rtime_training'],
        ptraining = r['timing']['ptime_training'],
        rtotal = r['timing']['rtime_total']))
print('\n')



List the training rounds :  dict_keys([0])

List the nodes for the last training round and their timings : 
	- config_mednist_2_sampled :    
		rtime_training=50.07 seconds    
		ptime_training=298.24 seconds    
		rtime_total=56.28 seconds




### Save and export your model 

In [84]:
#save model 
exp.training_plan().export_model('./training_plan2_densenet_MedNIST')

2024-02-06 14:09:00,979 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 14:09:06,668 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 14:09:48,746 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:09:59,963 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:10:06,661 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 14:10:15,469 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

### Tensorboard

In [82]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [83]:
%tensorboard --logdir "$tensorboard_dir"

Reusing TensorBoard on port 6006 (pid 45074), started 2:26:15 ago. (Use '!kill 45074' to kill it.)

2024-02-06 14:06:22,036 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:06:29,420 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:06:47,827 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 14:07:29,420 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:07:44,653 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:07:47,823 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 14:08:00,983 fedbiomed DEBUG - Node: config_mednist_2_sampled polling for the tasks

2024-02-06 14:08:44,648 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

2024-02-06 14:08:48,750 fedbiomed DEBUG - Node: config_mednist_1_sampled polling for the tasks

# Le reste est a nettoyer 

### You could import your first model from TrainingPlan1 instead of loading the original DenseNet  

In [ ]:
exp.training_plan().export_model('./training_plan2_densenet_MedNIST')

In [ ]:
# save your model ( all layers model of te training experiment)
remote_model = exp.training_plan().model()
torch.save(remote_model, './training_plan2_model')

In [ ]:
#from torchvision import models

#torch.save(models.densenet121(pretrained=True).state_dict(), './model_training_plan_2')

## Import your model and parameters 

In [ ]:
your_model = torch.load('./training_plan2_model')

In [ ]:
# load your parameters (tensors's values of your tuned-model)
tuned_model= torch.load('./training_plan2_densenet_MedNIST')

In [ ]:
# In a new TrainingPlan experiment you could import your tuned-model 
exp.training_plan().import_model('./training_plan2_densenet_MedNIST')

### This part needs confirmation, tests,( and agreements to load parameters ? ) 

In [ ]:
#remote_model = remote_experiment.training_plan().model()
tuned_model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])

In [ ]:
tuned_model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])
#tuned_model.load_state_dict(remote_experiment.aggregated_params()[rounds - 1]['params'])

In [ ]:
tuned_model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])